In [1]:
%useLatestDescriptors
%use lets-plot
%use dataframe

# Load data

In [10]:
import org.um.feri.ears.algorithms.gp.GPAlgorithmExecutor
import org.um.feri.ears.util.gp_stats.GPAlgorithmMultiConfigurationsProgressData
import org.um.feri.ears.util.gp_stats.GPProgramSolutionSimple

var dataFile = "C:\\Users\\marko\\UnityProjects\\GenIATraP_refactor\\GeneralTrainingEnvironmentForMAS\\evo_monitor\\src\\data\\multiConfigurationPrograssData.ser"
var progressData = GPAlgorithmMultiConfigurationsProgressData.deserializeState(dataFile)
System.out.println("Progress data loaded from file: " + progressData)

System.out.println("Data size: " +progressData.multiConfigurationProgressData.size)

Progress data loaded from file: org.um.feri.ears.util.gp_stats.GPAlgorithmMultiConfigurationsProgressData@27f3c90
Data size: 1


In [14]:
progressData.multiConfigurationProgressData[0].multiRunProgressData[0].gensProgressData[98].population.size

1200

# Prepare helper classes & functions

In [3]:
class SelectedIndividual(
    var selectedConfiguration: Int,
    var selectedRun: Int,
    var selectedGeneration: Int,
    var bestWorstIndividual: Int, // 0 = best, 1 = worst
) {
    constructor() : this(0, 0, 0, 0)
}

In [4]:
import org.um.feri.ears.individual.representations.gp.IndividualMatchResult

fun sumIndividualValues(individualMatchResult: IndividualMatchResult): Double {
    var sum = 0.0
    for (key in individualMatchResult.individualValues.keys) {
        sum += individualMatchResult.individualValues[key]!!
    }

    return sum
}

In [5]:
import org.yaml.snakeyaml.util.Tuple

// Prepare data (best, worst and average individual rating per generation)
fun prepareData(selectedConfiguration: Int, selectedRun: Int, bestGenIndividualsRating: MutableList<GPProgramSolutionSimple>, worstGenIndividualsRating: MutableList<GPProgramSolutionSimple>, avgGenIndividualRatings: MutableList<Double>, bestGenIndividualAvgFitnesses: MutableList<GPProgramSolutionSimple>, worstGenIndividualAvgFitnesses: MutableList<GPProgramSolutionSimple>, avgGenIndividualAvgFitnesses: MutableList<Double>){

    var multiConfigurationProgressData = progressData.multiConfigurationProgressData[selectedConfiguration]
    var multiRunProgressData = multiConfigurationProgressData.multiRunProgressData[selectedRun]

    for (genProgressData in multiRunProgressData.gensProgressData) {
        var bestGenIndividualRating = genProgressData.population[0]
        var worstIndividualRating = genProgressData.population[0]
        var avgGenIndividualRating = 0.0

        var bestGenIndividualAvgFitness = genProgressData.population[0]
        var worstIndividualAvgFitness = genProgressData.population[0]
        var avgGenIndividualAvgFitness = 0.0

        for (solution in genProgressData.population) {
            if (solution.finalIndividualFitness.additionalValues["Rating"]!! < bestGenIndividualRating.finalIndividualFitness.additionalValues["Rating"]!!) {
                bestGenIndividualRating = solution
            }

            if (solution.finalIndividualFitness.additionalValues["Rating"]!! > worstIndividualRating.finalIndividualFitness.additionalValues["Rating"]!!) {
                worstIndividualRating = solution
            }

            if(sumIndividualValues(solution.finalIndividualFitness.avgMatchResult) < sumIndividualValues(bestGenIndividualAvgFitness.finalIndividualFitness.avgMatchResult)) {
                bestGenIndividualAvgFitness = solution
            }

            if(sumIndividualValues(solution.finalIndividualFitness.avgMatchResult) > sumIndividualValues(worstIndividualAvgFitness.finalIndividualFitness.avgMatchResult)) {
                worstIndividualAvgFitness = solution
            }

            avgGenIndividualRating += solution.finalIndividualFitness.additionalValues["Rating"]!!
            avgGenIndividualAvgFitness += sumIndividualValues(solution.finalIndividualFitness.avgMatchResult)
        }

        bestGenIndividualsRating.add(bestGenIndividualRating)
        worstGenIndividualsRating.add(worstIndividualRating)

        avgGenIndividualRating /= genProgressData.population.size
        avgGenIndividualRatings.add(avgGenIndividualRating)

        bestGenIndividualAvgFitnesses.add(bestGenIndividualAvgFitness)
        worstGenIndividualAvgFitnesses.add(worstIndividualAvgFitness)

        avgGenIndividualAvgFitness /= genProgressData.population.size
        avgGenIndividualAvgFitnesses.add(avgGenIndividualAvgFitness)
    }
}

# Visualize data (line plot) - Best, worst and average individual rating per generation

In [73]:
// Input params for visualization START
val selectedConfiguration = 0
val selectedRun = 0
// Input params for visualization END

// Prepare data
val bestGenIndividualsRating = mutableListOf<GPProgramSolutionSimple>()
val worstGenIndividualsRating = mutableListOf<GPProgramSolutionSimple>()
val avgGenIndividualRatings = mutableListOf<Double>()

val bestGenIndividualAvgFitnesses = mutableListOf<GPProgramSolutionSimple>()
val worstGenIndividualAvgFitnesses = mutableListOf<GPProgramSolutionSimple>()
val avgGenIndividualAvgFitnesses = mutableListOf<Double>()

prepareData(selectedConfiguration, selectedRun, bestGenIndividualsRating, worstGenIndividualsRating, avgGenIndividualRatings, bestGenIndividualAvgFitnesses, worstGenIndividualAvgFitnesses, avgGenIndividualAvgFitnesses)

var multiConfigurationProgressData = progressData.multiConfigurationProgressData[selectedConfiguration]
var multiRunProgressData = multiConfigurationProgressData.multiRunProgressData[selectedRun]

var lastPhaseName = "";
val generations = mutableListOf<String>()
for (genProgressData in multiRunProgressData.gensProgressData) {
    generations.add(genProgressData.generation.toString() + genProgressData.executionPhaseName.substring(genProgressData.executionPhaseName.length - 6, genProgressData.executionPhaseName.length))
}

// Display data
val dfRating = dataFrameOf(
    "Generation" to generations + generations + generations,
    "Rating" to bestGenIndividualsRating.map { it.finalIndividualFitness.additionalValues["Rating"]!! } + worstGenIndividualsRating.map { it.finalIndividualFitness.additionalValues["Rating"]!! } + avgGenIndividualRatings,
    "Individual" to List(generations.size) { "Best" } + List(generations.size) { "Worst" } + List(generations.size) { "Average" }
)

val dfAvgFitness = dataFrameOf(
    "Generation" to generations + generations + generations,
    "Rating" to bestGenIndividualAvgFitnesses.map { sumIndividualValues(it.finalIndividualFitness.avgMatchResult) } + worstGenIndividualAvgFitnesses.map { sumIndividualValues(it.finalIndividualFitness.avgMatchResult) } + avgGenIndividualAvgFitnesses,
    "Individual" to List(generations.size) { "Best" } + List(generations.size) { "Worst" } + List(generations.size) { "Average" }
)

// The plot variable stores a plot with the temperature values for each city
val linePlotRating =
    letsPlot(dfRating.toMap()) { x = "Generation"; y = "Rating"; color = "Individual"} + ggsize(2000,400) + geomPoint(shape = 20) + geomLine() + ggtitle("Best Generation Rating")

val linePlotAvgFitness =
    letsPlot(dfAvgFitness.toMap()) { x = "Generation"; y = "Rating"; color = "Individual"} + ggsize(2000,800) + geomPoint(shape = 20) + geomLine() + ggtitle("Best Generation Average fitness")

val plots = listOf(linePlotRating, null, linePlotAvgFitness, null)
gggrid(plots, ncol = 2)

<path d="M0.5635430813428237 138.486938984075 L0.5635430813428237 138.486938984075 L3.3812584880569423 128.9960286900358 L6.198973894771061 124.63340546429924 L9.01668930148518 129.9633938413193 L11.834404708199298 124.37885958101327 L14.652120114913416 127.26338357124624 L17.469835521627537 145.38920283197876 L20.287550928341656 127.83066412083122 L23.105266335055774 138.46524169807313 L25.922981741769892 138.46524169807313 L28.74069714848401 139.37078293642728 L31.558412555198128 140.4618626744495 L34.376127961912246 150.33919829413804 L37.19384336862636 143.37315110147676 L40.01155877534048 152.7774558439285 L42.8292741820546 151.1627104392291 L45.64698958876872 145.74895384802184 L48.46470499548284 152.320130463713 L51.282420402196955 147.70616587693075 L54.10013580891108 147.70616587693075 L56.91785121562519 126.58599430226778 L59.73556662233931 120.33160140024793 L62.55328202905343 118.63220319970125 L65.37099743576755 132.7740936504419 L68.18871284248168 128.74844596653443 L71.00642824919579 130.70448968004325 L73.82414365590991 137.36262567998597 L76.64185906262404 136.71185063582178 L79.45957446933815 140.977817459486 L82.27728987605227 145.21490352645972 L85.09500528276638 140.00757825231827 L87.91272068948051 144.60327038624294 L90.73043609619462 139.1900478820757 L93.54815150290874 144.11733463090098 L96.36586690962287 153.46290982721166 L99.18358231633698 143.29809852017428 L102.0012977230511 149.1905606634189 L104.81901312976521 145.9744220300213 L107.63672853647934 147.1306203304011 L110.45444394319345 147.1306203304011 L113.27215934990757 133.01269035949448 L116.0898747566217 145.98625873404632 L118.90759016333581 139.99202964336493 L121.72530557004993 150.27408974591242 L124.54302097676404 153.13242344291913 L127.36073638347817 151.29292753568035 L130.17845179019227 147.10224695639863 L132.9961671969064 151.27513576115882 L135.81388260362053 145.51990060676604 L138.63159801033464 150.0282661715992 L141.44931341704876 143.20505388171026 L144.26702882376287 149.83858849737084 L147.08474423047699 147.00143802758993 L149.9024596371911 154.30288186726816 L152.72017504390524 154.5106951345501 L155.53789045061936 151.4799781692805 L158.35560585733347 154.60767198884645 L161.1733212640476 152.540077520936 L163.9910366707617 147.5592418702581 L166.80875207747582 152.2782980963014 L169.62646748418993 154.41955985725423 L172.44418289090407 154.07365170968038 L175.2618982976182 154.7619230020998 L178.0796137043323 152.33119274152997 L180.89732911104642 150.8752948366723 L183.71504451776053 152.46191722066925 L186.53275992447465 154.78695165601394 L189.35047533118876 151.77856954315027 L192.1681907379029 153.72951272542667 L194.98590614461702 151.3629163042344 L197.80362155133113 152.1542864236467 L200.62133695804525 153.0684064350816 L203.43905236475936 154.6473780222299 L206.25676777147348 151.40955879305042 L209.0744831781876 153.86075793943002 L211.89219858490173 152.95682230024 L214.70991399161585 147.68078339035256 L217.52762939832996 151.75257619452003 L220.34534480504408 148.26527155675905 L223.1630602117582 152.25577297538746 L225.9807756184723 152.70795776784252 L228.79849102518642 151.85935020795725 L231.61620643190057 154.13779222514592 L234.43392183861468 151.8598709428213 L237.2516372453288 154.33183939897066 L240.0693526520429 152.23868552814997 L242.88706805875702 146.98957795725693 L245.70478346547114 152.54456051402838 L248.52249887218525 148.0470803106802 L251.3402142788994 151.23751600527558 L254.1579296856135 146.77418399205033 L256.97564509232757 153.87904374446356 L259.7933604990417 154.48391734557978 L262.61107590575585 151.06254241486846 L265.42879131246997 154.55491754146593 L268.2465067191841 151.2924501953883 L271.0642221258982 155.49862931296337 L273.8819375326123 149.4897361949887 L276.6996529393264 147.45243113833686 " fill="none" stroke-width="1.6500000000000001" stroke="rgb(228,26,28)" stroke-opacity="1.0">
 
 
 
 <path d="M0.5635430813428237 48.2685723044278 L0.5635430813428237 48.26857230

# Visualize data (radar plot) - Comparison of individuals from different generations, runs & configurations

In [89]:
// Input params for visualization START
val selectedIndividuals = mutableListOf<SelectedIndividual>()
selectedIndividuals.add(SelectedIndividual(0, 0, 40, 0)) // Individual at position
selectedIndividuals.add(SelectedIndividual(0, 0, 90, 0))

val maxValues = mutableMapOf<String, Double>()
maxValues["SectorExploration"] = 5.0
maxValues["PowerUp_Pickup_Health"] = 10.0
maxValues["PowerUp_Pickup_Ammo"] = 15.0
maxValues["PowerUp_Pickup_Shield"] = 5.0
maxValues["MissilesFired"] = 20.0
maxValues["MissilesFiredAccuracy"] = 50.0
maxValues["SurvivalBonus"] = 5.0
maxValues["OpponentTrackingBonus"] = 5.0
maxValues["OpponentDestroyedBonus"] = 500.0
maxValues["DamageTakenPenalty"] = 50.0
// Input params for visualization END

// Prepare data for visualization
var selectedGPAlgorithmSolutionsSimple = mutableListOf<GPProgramSolutionSimple>()

// Get selected individuals
for (selectedIndividual in selectedIndividuals){
    // Prepare data
    val bestGenIndividualsRating = mutableListOf<GPProgramSolutionSimple>()
    val worstGenIndividualsRating = mutableListOf<GPProgramSolutionSimple>()
    val avgGenIndividualRatings = mutableListOf<Double>()

    val bestGenIndividualAvgFitnesses = mutableListOf<GPProgramSolutionSimple>()
    val worstGenIndividualAvgFitnesses = mutableListOf<GPProgramSolutionSimple>()
    val avgGenIndividualAvgFitnesses = mutableListOf<Double>()

    prepareData(selectedIndividual.selectedConfiguration, selectedIndividual.selectedRun, bestGenIndividualsRating, worstGenIndividualsRating, avgGenIndividualRatings, bestGenIndividualAvgFitnesses, worstGenIndividualAvgFitnesses, avgGenIndividualAvgFitnesses)

    var individual: GPProgramSolutionSimple?;
    if(selectedIndividual.bestWorstIndividual == 0) {
        individual = bestGenIndividualsRating[selectedIndividual.selectedGeneration]
    }
    else {
        individual = worstGenIndividualsRating[selectedIndividual.selectedGeneration]
    }

    selectedGPAlgorithmSolutionsSimple.add(individual!!)
}

// Find all fitness keys
val fitnessKeys = mutableListOf<String>()
for (individual in selectedGPAlgorithmSolutionsSimple) {
    for (key in individual.finalIndividualFitness.avgMatchResult.individualValues.keys) {
        if (key !in fitnessKeys) {
            fitnessKeys.add(key)
        }
    }
}

// Order fitnessKeys
fitnessKeys.sort()

// Build labels for visualization
val labels = mutableListOf<String>()
for (individual in selectedGPAlgorithmSolutionsSimple) {
    labels.addAll(fitnessKeys)
}

val datasets = mutableListOf<MutableList<Double>>()

// Build datasets for visualization
for (individual in selectedGPAlgorithmSolutionsSimple) {
    val dataset = mutableListOf<Double>()
    for (key in fitnessKeys) {
        if(individual.finalIndividualFitness.avgMatchResult.individualValues[key] == null) {
            dataset.add(0.0)
            System.out.println(key + ": 0.0")
            continue
        }
        dataset.add(Math.abs(individual.finalIndividualFitness.avgMatchResult.individualValues[key]!!) / maxValues[key]!!)
        System.out.println(key + ": " + individual.finalIndividualFitness.avgMatchResult.individualValues[key]!!)
    }
    System.out.println("/////////////////////////////")
    datasets.add(dataset)
}

var fitnessKeysIndexed = mutableListOf<Int>()
for (i in 0 until fitnessKeys.size) {
    fitnessKeysIndexed.add(i + 1)
}

var individualValueIdsMapped = mutableListOf<Int>()
var individualMapped = mutableListOf<String>()
var index = 0
for(dataset in datasets) {
    individualValueIdsMapped.addAll(fitnessKeysIndexed)
    individualMapped.addAll(List(fitnessKeys.size) { "Individual " + (index + 1) })
    index++
}

// Build map for visualization
val selectedIndividualsData = mapOf(
    "IndividualValueLabels" to labels,
    "IndividualValueId" to individualValueIdsMapped,
    "Individual" to individualMapped,
    "IndividualValues" to datasets.map { it }.toMutableList().flatten()
)

// Map each fitnessKey to a map 1 -> "fitnessKey1", 2 -> "fitnessKey2", ...
var labelsIndexed = fitnessKeys.mapIndexed { index, s -> index to s }.toMap().mapKeys { it.key + 1 }

letsPlot(selectedIndividualsData) +
        geomArea(flat = true) {   // <-- flat. I.e., do not transform segments to curves
            x = "IndividualValueId"; y = "IndividualValues"
        } +
        geomPoint() { x = "IndividualValueId"; y = "IndividualValues"; color="Individual" } +
        scaleXDiscrete(labels = labelsIndexed) +
        coordPolar()

DamageTakenPenalty: 0.0
MissilesFired: -0.54545456
MissilesFiredAccuracy: 0.0
OpponentDestroyedBonus: 0.0
OpponentTrackingBonus: 0.0
PowerUp_Pickup_Ammo: -2.7272727
PowerUp_Pickup_Health: 0.0
PowerUp_Pickup_Shield: -1.3636364
SectorExploration: -1.6363454
SurvivalBonus: -5.0
/////////////////////////////
DamageTakenPenalty: 13.708826
MissilesFired: -2.8363633
MissilesFiredAccuracy: -15.343746
OpponentDestroyedBonus: -208.33331
OpponentTrackingBonus: -3.6407273
PowerUp_Pickup_Ammo: -6.166199
PowerUp_Pickup_Health: -0.45454547
PowerUp_Pickup_Shield: -0.09090909
SectorExploration: -3.242427
SurvivalBonus: -4.8097277
/////////////////////////////


<path d="M170.5 100.04545454545456 L177.4057530686736 100.38471243900432 L184.24499996022723 101.39921888068147 L190.95187498838257 103.0792036188671 L197.46178728026769 105.40848748215933 L203.71204282183166 108.36463819363863 L209.642448235472 111.91918640595705 L215.19589047516592 116.03789987671627 L220.31888685632495 120.68111314367508 L224.96210012328373 125.80410952483408 L229.08081359404295 131.357551764528 L232.63536180636137 137.28795717816834 L235.59151251784067 143.53821271973231 L237.9207963811329 150.04812501161743 L239.60078111931853 156.7550000397728 L240.61528756099568 163.5942469313264 L240.95454545454544 170.5 L240.61528756099568 177.4057530686736 L239.60078111931853 184.2449999602272 L237.9207963811329 190.95187498838257 L235.59151251784067 197.46178728026769 L232.63536180636137 203.71204282183166 L229.08081359404298 209.64244823547196 L224.96210012328373 215.19589047516592 L220.31888685632495 220.31888685632492 L215.19589047516592 224.96210012328373 L209.64244823547196 229.08081359404298 L203.71204282183166 232.63536180636137 L197.46178728026769 235.59151251784067 L190.95187498838257 237.9207963811329 L184.24499996022723 239.60078111931853 L177.4057530686736 240.61528756099568 L170.5 240.95454545454544 L163.5942469313264 240.61528756099568 L156.75500003977277 239.60078111931853 L150.04812501161743 237.9207963811329 L143.53821271973231 235.59151251784067 L137.28795717816834 232.63536180636137 L131.35755176452804 229.08081359404298 L125.80410952483408 224.96210012328373 L120.68111314367506 220.31888685632495 L116.03789987671628 215.19589047516595 L111.91918640595703 209.642448235472 L108.36463819363863 203.71204282183166 L105.40848748215936 197.4617872802677 L103.07920361886711 190.95187498838257 L101.39921888068149 184.24499996022723 L100.3847124390043 177.40575306867356 L100.04545454545455 170.5 L100.3847124390043 163.59424693132644 L101.39921888068149 156.75500003977277 L103.0792036188671 150.04812501161746 L105.40848748215936 143.5382127197323 L108.36463819363863 137.28795717816834 L111.91918640595702 131.35755176452804 L116.03789987671627 125.80410952483408 L120.68111314367505 120.68111314367508 L125.80410952483405 116.03789987671627 L131.357551764528 111.91918640595705 L137.28795717816834 108.36463819363863 L143.53821271973226 105.40848748215936 L150.04812501161743 103.0792036188671 L156.75500003977277 101.3992188806815 L163.5942469313264 100.38471243900432 L170.5 100.04545454545456 " stroke="rgb(233,233,233)" stroke-opacity="1.0" stroke-width="1.0" fill="none">
 
 <path d="M170.5 29.590909090909122 L184.3115061373472 30.269424878008635 L197.98999992045444 32.29843776136295 L211.40374997676514 35.658407237734195 L224.42357456053537 40.31697496431866 L236.92408564366332 46.22927638727725 L248.78489647094398 53.3383728119141 L259.89178095033185 61.575799753432534 L270.1377737126499 70.86222628735015 L279.42420024656747 81.10821904966815 L287.6616271880859 92.21510352905605 L294.77072361272275 104.07591435633668 L300.68302503568134 116.57642543946463 L305.3415927622658 129.59625002323486 L308.70156223863705 143.01000007954556 L310.73057512199136 156.68849386265282 L311.4090909090909 170.5 L310.73057512199136 184.31150613734718 L308.70156223863705 197.98999992045444 L305.3415927622658 211.40374997676514 L300.68302503568134 224.42357456053537 L294.77072361272275 236.92408564366332 L287.66162718808596 248.78489647094392 L279.42420024656747 259.89178095033185 L270.1377737126499 270.13777371264985 L259.89178095033185 279.42420024656747 L248.78489647094392 287.66162718808596 L236.92408564366332 294.77072361272275 L224.42357456053537 300.68302503568134 L211.40374997676514 305.3415927622658 L197.98999992045447 308.70156223863705 L184.3115061373472 310.73057512199136 L170.5 311.4090909090909 L156.68849386265282 310.73057512199136 L143.01000007954553 308.70156223863705 L129.5962500232349 305.3415927622658 L116.57642543946463 300.68302503568134 L104.0759143563367 294.77072361272275 L92.2151035290561 287.661627188085

Ž# Selected Individual Details

In [57]:
import org.jetbrains.letsPlot.intern.Plot

// Input params for visualization START
val selectedIndividuals = mutableListOf<SelectedIndividual>()
selectedIndividuals.add(SelectedIndividual(0, 0, 0, 0)) // Individual at position
selectedIndividuals.add(SelectedIndividual(0, 0, 98, 0))
// Input params for visualization END

// Prepare data for visualization
var selectedGPAlgorithmSolutionsSimple = mutableListOf<GPProgramSolutionSimple>()

// Get selected individuals
for (selectedIndividual in selectedIndividuals){
    // Prepare data
    val bestGenIndividualsRating = mutableListOf<GPProgramSolutionSimple>()
    val worstGenIndividualsRating = mutableListOf<GPProgramSolutionSimple>()
    val avgGenIndividualRatings = mutableListOf<Double>()

    val bestGenIndividualAvgFitnesses = mutableListOf<GPProgramSolutionSimple>()
    val worstGenIndividualAvgFitnesses = mutableListOf<GPProgramSolutionSimple>()
    val avgGenIndividualAvgFitnesses = mutableListOf<Double>()

    prepareData(selectedIndividual.selectedConfiguration, selectedIndividual.selectedRun, bestGenIndividualsRating, worstGenIndividualsRating, avgGenIndividualRatings, bestGenIndividualAvgFitnesses, worstGenIndividualAvgFitnesses, avgGenIndividualAvgFitnesses)

    var individual: GPProgramSolutionSimple?;
    if(selectedIndividual.bestWorstIndividual == 0) {
        individual = bestGenIndividualsRating[selectedIndividual.selectedGeneration]
    }
    else {
        individual = worstGenIndividualsRating[selectedIndividual.selectedGeneration]
    }

    selectedGPAlgorithmSolutionsSimple.add(individual!!)
}

var plots = mutableListOf<Plot>()

var index = 0
for (individual in selectedGPAlgorithmSolutionsSimple) {
    var nodeCountsData = mapOf(
        "NodeName" to individual.nodeCounts.keys.toList(),
        "NodeCount" to individual.nodeCounts.values.toList()
    )

    val blankTheme = theme(line=elementBlank(), axis=elementBlank())
    val (w, h) = 800 to 400
    val p = letsPlot(nodeCountsData) + ggsize(w,h) + blankTheme

    val plot = p + geomPie(stat = Stat.identity,
            size = 20, stroke = 1, color = "white", hole = 0.5) { slice = "NodeCount"; fill = "NodeName" } +
            blankTheme +
            scaleFillBrewer(palette = "Set1") + ggtitle("Conf: " + selectedIndividuals[index].selectedConfiguration + " Run: " + selectedIndividuals[index].selectedRun + " Gen: " + selectedIndividuals[index].selectedGeneration + " IndividualID: " + individual.individualId)

    System.out.println(
        "Individual: " + "(Conf: " + selectedIndividuals[index].selectedConfiguration + " Run: " + selectedIndividuals[index].selectedRun + " Gen: " + selectedIndividuals[index].selectedGeneration + " IndividualID: " + individual.individualId + ")\nChangesCount: " + individual.changesCount + "\nFunctionNodesCount: " + individual.functionNodes + "\nTerminalNodesCount: " + individual.terminalNodes + "\nTreeSize: " + individual.treeSize + "\nTreeDepth: " + individual.treeDepth + "\nFitness: " + (individual.finalIndividualFitness.additionalValues["Rating"].toString() + ", " + individual.finalIndividualFitness.additionalValues["StdDeviation"]) + "\nIndividualAvgFitness: " + sumIndividualValues(individual.finalIndividualFitness.avgMatchResult) + "\nIndividualAvgFitnessValues: " + individual.finalIndividualFitness.avgMatchResult.individualValues.map { it.key + ": " + it.value }.toString()
        + "\n\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"
    )

    index++

    plots.add(plot)
}

gggrid(plots, ncol = 2)

Individual: (Conf: 0 Run: 0 Gen: 0 IndividualID: 688)
ChangesCount: 2
FunctionNodesCount: 6
TerminalNodesCount: 13
TreeSize: 19
TreeDepth: 4
Fitness: -44.413055, 2.8863385
IndividualAvgFitness: -3.4848366
IndividualAvgFitnessValues: [SectorExploration: -3.4848366]
\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
Individual: (Conf: 0 Run: 0 Gen: 98 IndividualID: 13379)
ChangesCount: 48
FunctionNodesCount: 68
TerminalNodesCount: 91
TreeSize: 159
TreeDepth: 11
Fitness: -47.098907, 2.7834136
IndividualAvgFitness: -142.431787
IndividualAvgFitnessValues: [OpponentDestroyedBonus: -113.63636, SectorExploration: -3.4242547, SurvivalBonus: -4.857909, DamageTakenPenalty: 2.8012726, OpponentTrackingBonus: -3.2165453, PowerUp_Pickup_Ammo: -6.344345, PowerUp_Pickup_Shield: -0.6742454, MissilesFired: -2.6636362, MissilesFiredAccuracy: -10.415764]
\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\


Conf: 0 Run: 0 Gen: 0 IndividualID: 688 
 
 
 
 
 
 
 
 
 NodeName 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 RayHitObject 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Sequencer 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Rotate 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 MoveForward 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Selector 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Conf: 0 Run: 0 Gen: 98 IndividualID: 13379 
 
 
 
 
 
 
 
 
 NodeName 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 RayHitObject 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Inverter 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Sequencer 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Rotate 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ShieldLevelBellow 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 HealthLevelBellow 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 AmmoLevelBellow 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 MoveForward 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Shoot 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Selector

# Convergence graphs

In [58]:
// Input params for visualization START
val selectedConfiguration = 0
val selectedRun = 0
// Input params for visualization END

progressData.multiConfigurationProgressData[selectedConfiguration].multiRunProgressData[selectedRun].convergenceGraphData

val n = 3
val Generation = mutableListOf<Int>()
val Rating = mutableListOf<Double>()

val convergenceDataGraph = progressData.multiConfigurationProgressData[selectedConfiguration].multiRunProgressData[selectedRun].convergenceGraphData;
for (i in 0 until convergenceDataGraph.size) {
    Generation.addAll(List(n) {i + 1})
    val rating = Math.abs(convergenceDataGraph[i].finalIndividualFitness.additionalValues["Rating"]!!)
    Rating.addAll(listOf(rating - (2*convergenceDataGraph[i].finalIndividualFitness.additionalValues["StdDeviation"]!!), rating, rating + (2*convergenceDataGraph[i].finalIndividualFitness.additionalValues["StdDeviation"]!!)))
}

val data = mapOf<String, Any>(
    "Generation" to Generation,
    "Rating" to Rating
)

letsPlot(data) { x = "Generation"; y = "Rating"}  + geomBoxplot(whiskerWidth = 0.2) + ggtitle("Convergence graph (Tournament between generation best individuals)")

-0.4 
 
 
 
 
 
 
 
 
 -0.2 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 0.2 
 
 
 
 
 
 
 
 
 0.4 
 
 
 
 
 
 
 
 
 
 
 -0.4 
 
 
 
 
 
 
 -0.2 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 0.2 
 
 
 
 
 
 
 0.4 
 
 
 
 
 
 
 
 
 Convergence graph (Tournament between generation best individuals) 
 
 
 
 
 y 
 
 
 
 
 x